# Check TCdata360 Data Sources (if updated)

In [244]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import datetime
import re
import io

%matplotlib inline

In [245]:
date_today = datetime.date.today().isoformat()

In [3]:
tc_datasets = 'http://tcdata360-backend.worldbank.org/api/v1/datasets'
df_tc_datasets = pd.read_json(requests.get(tc_datasets).text)

In [246]:
df_answer_key = pd.read_csv('2017-07-28-Check-TC-Data-Update-Answer-Key.csv', encoding='latin-1')

In [247]:
df_answer_key.head()

,slug,title,source_name,source_link,legal_text,legal_link,id,Dataset / sub-dataset,Site,Data Coverage,Start Year,Latest Year
0,itip_goods,I-TIP Goods,WTO I-TIP Goods,https://i-tip.wto.org/goods,World Trade Organization,https://www.wto.org/english/info_e/cont_e.htm#...,44,WTO Integrated Trade Intelligence Portal (I-TI...,tc,2005-2016,2005,2016
1,itip_goods,I-TIP Goods,WTO I-TIP Goods,https://i-tip.wto.org/goods,World Trade Organization,https://www.wto.org/english/info_e/cont_e.htm#...,44,WTO Integrated Trade Intelligence Portal (I-TIP),tc,2005-2016,2005,2016
2,escap,ESCAP,ESCAP,http://data.worldbank.org/data-catalog/trade-c...,World Bank Group,http://www.worldbank.org/terms-datasets,45,ESCAP Trade Costs Database,tc,1995 - 2015,1995,2015
3,innovation_indicators,OECD Innovation Indicators,OECD,http://www.oecd.org/innovation/inno/inno-stats...,Organization for Economic Co-operation and Dev...,http://www.oecd.org/termsandconditions/,46,OECD Innovation Indicators,tc,"2013, 2015",2013,2015
4,investing_across_borders,Investing Across Borders,WBG - Investing Across Borders,http://iab.worldbank.org,World Bank Group,http://www.worldbank.org/terms-datasets,47,Investing Across Borders,tc,"2010, 2012",2010,2012


In [277]:
df_answer_key.columns

Index(['slug', 'title', 'source_name', 'source_link', 'legal_text',
       'legal_link', 'id', 'Dataset / sub-dataset', 'Site', 'Data Coverage',
       'Start Year', 'Latest Year'],
      dtype='object')

Loop through all dataset IDs and generate key statistics per dataset:
- number of unique countries
- number of unique indicators
- current start year, excluding 100% NULL columns
- end start year, excluding 100% NULL columns
- whole timeframe (as a list), excluding 100% NULL columns

## Scratch

In [94]:
data_columns = ['title', 'id', 'number_of_unique_countries', 'number_of_unique_indicators', 'number_of_unique_subindicators',
               'start_year', 'end_year', 'timeframe']

In [95]:
tc_data_check = pd.DataFrame()

In [57]:
# for row in df_tc_datasets.index:
#     id_dataset = df_tc_datasets['id'].ix[row]
#     title_dataset = df_tc_datasets['title'].ix[row]

In [31]:
import random
id_dataset = random.choice(list(df_tc_datasets['id']))
dat_url = "http://tcdata360-backend.worldbank.org/api/v1/datasets/" + str(id_dataset) + "/dump.csv"
dat = requests.get(dat_url)
df_dat = pd.read_csv(io.StringIO(dat.text))

Drop all completely NULL columns and rows

In [42]:
df_dat.dropna(axis=1, how='all', inplace=True)
df_dat.dropna(axis=0, how='all', inplace=True)

Generate features

In [99]:
num_of_countries = len(df_dat['Country ISO3'].value_counts().index)
num_of_indicators = len(df_dat['Indicator'].value_counts().index)
num_of_subindicators = df_dat[['Indicator', 'Subindicator Type']].drop_duplicates().shape[0]

In [100]:
list_timeframe = [col for col in df_dat.columns if col not in ['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type']]
list_timeframe = sorted(list_timeframe)

In [101]:
start_year = list_timeframe[0]
end_year = list_timeframe[-1]

Compile all data

In [103]:
dataset_list = [title_dataset, id_dataset, num_of_countries, num_of_indicators, num_of_subindicators, start_year, end_year, list_timeframe]

In [104]:
df_temp = pd.DataFrame(dataset_list).T

In [105]:
tc_data_check = tc_data_check.append(pd.DataFrame(dataset_list).T)

In [106]:
tc_data_check.columns = data_columns

In [108]:
tc_data_check.reset_index(drop=True, inplace=True)

## Compiling into loop

In [129]:
data_columns = ['title', 'id', 'number_of_unique_countries', 'number_of_unique_indicators', 'number_of_unique_subindicators',
               'start_year', 'end_year', 'timeframe']
tc_data_check = pd.DataFrame()

success_counter = 0
error_counter = 0
error_list = []
total_count = len(df_tc_datasets.index)

for row in df_tc_datasets.index:
    id_dataset = df_tc_datasets['id'].ix[row]
    title_dataset = df_tc_datasets['title'].ix[row]
    
    dat_url = "http://tcdata360-backend.worldbank.org/api/v1/datasets/" + str(id_dataset) + "/dump.csv"
    dat = requests.get(dat_url)
    df_dat = pd.read_csv(io.StringIO(dat.text))
    
    print(df_dat.columns) #for debugging
    
    #drop all completele NULL columns and rows
    df_dat.dropna(axis=1, how='all', inplace=True)
    df_dat.dropna(axis=0, how='all', inplace=True)
    
    try:
        # Generate features for checking
        num_of_countries = len(df_dat['Country ISO3'].value_counts().index)
        num_of_indicators = len(df_dat['Indicator'].value_counts().index)
        num_of_subindicators = df_dat[['Indicator', 'Subindicator Type']].drop_duplicates().shape[0]

        nondate_cols = ['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type', 'Partner', 'MRV', 'Product']
        list_timeframe = [col for col in df_dat.columns if col not in nondate_cols]
        list_timeframe = sorted(list_timeframe)
        start_year = list_timeframe[0]
        end_year = list_timeframe[-1]

        # compile all data
        dataset_list = [title_dataset, id_dataset, num_of_countries, num_of_indicators, num_of_subindicators, start_year, end_year, list_timeframe]
        print(dataset_list) #for debugging

        df_temp = pd.DataFrame(dataset_list).T
        tc_data_check = tc_data_check.append(pd.DataFrame(dataset_list).T)
        
    except:
        error_counter += 1
        error_list.append([title_dataset, id_dataset])
        continue
        
    success_counter += 1
    
    print("Done checking %d out of %d datasets." % (success_counter + error_counter, total_count))
    
# Clean up final dataframe
tc_data_check.columns = data_columns
tc_data_check.reset_index(drop=True, inplace=True)

print("Done with all %d datasets with %d successes and %d failures." % (total_count, success_counter, error_counter))
print("Failed datasets include %s" % str(error_list))

Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015'],
      dtype='object')
['PEFA', 590, 71, 111, 111, '2005', '2015', ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
Done checking 1 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')
['Enterprise Surveys', 49, 147, 121, 121, '2002', '2016', ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']]
Done checking 2 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2007', '2010', '2012', '2014', '2016'],
      dtype='object')
['Logistics Performance Index', 50, 167, 7,

Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       'Partner', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014'],
      dtype='object')
['ESCAP', 45, 192, 3, 3, '1995', '2014', ['1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']]
Done checking 13 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type', 'MRV',
       '2011', '2014'],
      dtype='object')
['WB: Financial Inclusion Index (Findex)', 66, 155, 6, 6, '2011', '2014', ['2011', '2014']]
Done checking 14 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')
['WEF GCI Most Problematic Factors', 71, 1

C:\Users\mrpso\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       'Product', 'Partner', '1948', '1949', '1950', '1950-1960',
       ...
       '2014', '2014Q1', '2014Q2', '2014Q3', '2014Q4', '2015', '2015Q1',
       '2015Q2', '2015Q3', '2015Q4'],
      dtype='object', length=156)
['United Nations Conference on Trade and Development Statistics', 76, 207, 165, 323, '1948', '2015Q4', ['1948', '1949', '1950', '1950-1960', '1950-1970', '1950-1980', '1950-1990', '1950-2000', '1950-2010', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1960-1970', '1960-1980', '1960-1990', '1960-2000', '1960-2010', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1970-1980', '1970-1990', '1970-2000', '1970-2005', '1970-2010', '1971', '1972', '1973', '1974', '1975', '1975-2005', '1976', '1977', '1978', '1979', '1980', '1980-1985', '1980-1990', '1980-2000', '1980-2010', '1981', '1982', '1983', '1984', '1985', '1985-1990', '1985-2005'

Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014'],
      dtype='object')
['Fiscal Rules Dataset', 98, 88, 60, 60, '1987', '2014', ['1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']]
Done checking 37 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016'],
      dtype='object')
['Country Policy and Institutional Assessments', 102, 85, 16, 16, '2005', '2016', ['2005', '2006', '2007', '2008', '2009', '20

Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2006', '2008', '2010', '2012', '2014', '2016'],
      dtype='object')
['Bertelsmann Transformation Index', 67, 128, 3, 3, '2006', '2016', ['2006', '2008', '2010', '2012', '2014', '2016']]
Done checking 55 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015'],
      dtype='object')
['Polity IV', 89, 78, 16, 16, '1987', '2015', ['1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
Done checking 56 out of 76 datasets.
Index(['Country ISO3'

Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '2024', '2025', '2026', '2027'],
      dtype='object')
['World Travel & Tourism Council', 79, 176, 12, 66, '1995', '2027', ['1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027']]
Done checking 59 out of 76 datasets.
Index(['Country ISO3', 'Country Name', 'Indicator', 'Subindicator Type',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object')
['Global Competitiveness Index', 430, 148, 108, 108

In [131]:
tc_data_check.head()

,title,id,number_of_unique_countries,number_of_unique_indicators,number_of_unique_subindicators,start_year,end_year,timeframe
0,PEFA,590,71,111,111,2005,2015,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201..."
1,Enterprise Surveys,49,147,121,121,2002,2016,"[2002, 2003, 2004, 2005, 2006, 2007, 2008, 200..."
2,Logistics Performance Index,50,167,7,20,2007,2016,"[2007, 2010, 2012, 2014, 2016]"
3,Worldwide Governance Indicators,51,205,6,30,1996,2015,"[1996, 1998, 2000, 2002, 2003, 2004, 2005, 200..."
4,World Economic Forum Global Competitiveness Index,53,151,120,240,2007-2008,2016-2017,"[2007-2008, 2008-2009, 2009-2010, 2010-2011, 2..."


# Merge against TC datasets

In [141]:
df_tc_data_check = df_tc_datasets[['slug', 'id', 'source','title']].merge(tc_data_check, how="outer", on=['id', 'title'])

## Confirming if errors are valid

In [143]:
df_tc_data_check[df_tc_data_check['number_of_unique_countries'].isnull()]

,slug,id,source,title,number_of_unique_countries,number_of_unique_indicators,number_of_unique_subindicators,start_year,end_year,timeframe
42,ArabBarAll,104,ArabBarometer,ArabBarometer,NaN,NaN,NaN,NaN,NaN,NaN
61,DAGI,101,"Development, Aid and Governance Indicators","Development, Aid and Governance Indicators",NaN,NaN,NaN,NaN,NaN,NaN
62,AfroBarAll,105,AfroBarometer,AfroBarometer,NaN,NaN,NaN,NaN,NaN,NaN
72,GCB,96,Transparency - Global Corruption Barometer,Global Corruption Barometer,NaN,NaN,NaN,NaN,NaN,NaN
74,LatinBar,90,Latinobarometro,Latinobarometro,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning dates for comparison

In [164]:
def clean_up_dates(x):
    cleaned_date = x
    m = re.match('(\d{4})-(\d{4})', str(x))
    if m:
        cleaned_date = m.groups()[-1]
    else:
        n = re.match('(\d{4})Q\d{1}', str(x))
        if n:
            cleaned_date = n.groups()[-1]
        else:
            o = re.match('(\d{4})M\d{2}', str(x))
            if o:
                cleaned_date = o.groups()[-1]
    return cleaned_date

In [170]:
df_tc_data_check['start_year-clean'] = df_tc_data_check['start_year'].apply(lambda x: clean_up_dates(x))
df_tc_data_check['end_year-clean'] = df_tc_data_check['end_year'].apply(lambda x: clean_up_dates(x))

In [171]:
df_tc_data_check.head()

,slug,id,source,title,number_of_unique_countries,number_of_unique_indicators,number_of_unique_subindicators,start_year,end_year,timeframe,start_year-clean,end_year-clean
0,PEFA,590,PEFA,PEFA,71,111,111,2005,2015,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",2005,2015
1,enterprise_surveys,49,WBG - Enterprise Surveys,Enterprise Surveys,147,121,121,2002,2016,"[2002, 2003, 2004, 2005, 2006, 2007, 2008, 200...",2002,2016
2,logistics_performance_index,50,WBG - Logistics Performance Index,Logistics Performance Index,167,7,20,2007,2016,"[2007, 2010, 2012, 2014, 2016]",2007,2016
3,worldwide_governance_indicators,51,Worldwide Governance Indicators,Worldwide Governance Indicators,205,6,30,1996,2015,"[1996, 1998, 2000, 2002, 2003, 2004, 2005, 200...",1996,2015
4,global_competitiveness_index,53,World Economic Forum Global Competitiveness Index,World Economic Forum Global Competitiveness Index,151,120,240,2007-2008,2016-2017,"[2007-2008, 2008-2009, 2009-2010, 2010-2011, 2...",2008,2017


In [264]:
df_tc_data_check['id'] = df_tc_data_check['id'].astype(str)

In [265]:
df_merge = df_tc_data_check.merge(df_answer_key, how='outer', on=['id', 'slug', 'title'])

## Removing datasets which are/have:
- without data
- new datasets (with data yet to be ingested)
- duplicated (same slug as another's)

In [267]:
df_merge_shortlist = df_merge[['id', 'title', 'slug', 'Site', 'start_year-clean', 'end_year-clean', 'Start Year', 'Latest Year']].dropna(axis=0, how='any', subset=['id','start_year-clean','Start Year'])

In [269]:
df_merge_shortlist = df_merge_shortlist[(df_merge_shortlist['Start Year'] != 'N/A (duplicate dataset)') & (df_merge_shortlist['Start Year'] != 'N/A (too general)')]

In [270]:
df_merge_shortlist['start_year-check'] = df_merge_shortlist['start_year-clean'] <= df_merge_shortlist['Start Year']
df_merge_shortlist['end_year-check'] = df_merge_shortlist['end_year-clean'] >= df_merge_shortlist['Latest Year']

In [271]:
def amida_action(x):
    action = "Updated. No action required."
    
    end = x['end_year-check']
    start = x['start_year-check']
    
    if (not end) & (not start):
        action = "Not updated (both start and end dates)"
    elif end & (not start):
        action = "Not updated (start date only)"
    elif (not end) & (start):
        action = "Not updated (end date only)"
    
    if (x['Site'] == 'gv') & (action != "Updated. No action required."):
        action = "[Gov dataset - For Luis' action] "+ action
    
    return action

In [272]:
df_merge_shortlist

,id,title,slug,Site,start_year-clean,end_year-clean,Start Year,Latest Year,start_year-check,end_year-check
0,590,PEFA,PEFA,gv,2005,2015,2005,2016,True,False
2,50,Logistics Performance Index,logistics_performance_index,tc,2007,2016,2007,2016,True,True
5,147,Public Investment Management,PIM,gv,2010,2010,1987,2016,False,False
6,483,Human Resource Management,HRM,gv,2009,2009,1987,2016,False,False
7,512,UNCTAD Cyberlaw Tracker,UNCTAD Cyberlaw Tracker,tc,2016,2016,2016,2016,True,True
8,479,Pay and Employment,PAY_EMP,gv,1990,2008,1987,2016,False,False
9,44,I-TIP Goods,itip_goods,tc,1990,2016,2005,2016,True,True
10,44,I-TIP Goods,itip_goods,tc,1990,2016,2005,2016,True,True
12,46,OECD Innovation Indicators,innovation_indicators,tc,1919,2016,2013,2015,True,True
13,45,ESCAP,escap,tc,1995,2014,1995,2015,True,False


In [273]:
df_merge_shortlist['amida_action'] = df_merge_shortlist.apply(lambda x: amida_action(x), axis=1)

In [274]:
df_merge_shortlist.head()

,id,title,slug,Site,start_year-clean,end_year-clean,Start Year,Latest Year,start_year-check,end_year-check,amida_action
0,590,PEFA,PEFA,gv,2005,2015,2005,2016,True,False,[Gov dataset - For Luis' action] Not updated (...
2,50,Logistics Performance Index,logistics_performance_index,tc,2007,2016,2007,2016,True,True,Updated. No action required.
5,147,Public Investment Management,PIM,gv,2010,2010,1987,2016,False,False,[Gov dataset - For Luis' action] Not updated (...
6,483,Human Resource Management,HRM,gv,2009,2009,1987,2016,False,False,[Gov dataset - For Luis' action] Not updated (...
7,512,UNCTAD Cyberlaw Tracker,UNCTAD Cyberlaw Tracker,tc,2016,2016,2016,2016,True,True,Updated. No action required.


In [275]:
df_merge_shortlist.columns = ['id', 'title', 'slug', 'Site', 'current-start_year', 'current-end_year',
       'ideal-start_year', 'ideal-end_year', 'start_year-check', 'end_year-check',
       'amida_action']

In [279]:
df_merge_shortlist.to_csv("%s-Check-TC-Data-Update.csv" % date_today, index=False)
df_merge.to_csv("%s-Check-TC-Data-Update-verboseprofile.csv" % date_today, index=False)